In [185]:
# import all the required libraries
import requests
import json
import pandas as pd
import time
from pandas import json_normalize
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException,NoSuchElementException,ElementClickInterceptedException

# url you want to extract reviews and product information from
url = 'https://www.flipkart.com/search?q=smart+phones&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&sort=recency_desc'

# define the driver variable
# change the executable_path with the path of your chrome driver
driver = webdriver.Chrome(executable_path=r'C:\Users\Priyanka\anaconda3\Lib\site-packages\chromedriver.exe')

# initiate the process
driver.get(url)
# since it takes some time to load any website, put the code to sleep for sometime
time.sleep(5)

# initiate your list where you want to store data of different products
my_list_phone = []

# get the class name of the object you want to click using inspect 
for item in driver.find_elements_by_class_name('_3wU53n')[5:]:
    item.click()
    
    handles = driver.window_handles        # get all the tabs that are opened
    driver.switch_to.window(handles[1])    # switch to the tab you want to scrape data from
    
    time.sleep(5)                          #give time to load
    
    # get the source code of the page using BeautifulSoup
    start_soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # define variables to store the scrapped data
    Title = start_soup.find('span', {"class":'_35KyD6'}).text
    Price = start_soup.find('div', {"class": "_1vC4OE"}).text
    Ratings = start_soup.find('div', {"class": "ebepc-"}).get_text(separator=u' ')
    Specifications = ''
    for item in start_soup.find_all('tr', {"class": "_3_6Uyw row"}):
        str(item.getText(separator=u' - '))
        Specifications = Specifications+'\n'+str(item.getText(separator=u' - '))
    
    # click on all reviews to get all the revies
    # WebDriverWait will allow to wait before the item is clickable
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "swINJg"))).click()
    time.sleep(5)
    
    # initiate your reviews variable
    Reviews = ''
    
    # start loop to extract reviews from all the pages 
    while True:
        
        try:
            # since all the reviews don't have read more option so ignore those that doesn't have read more
            for item in driver.find_elements(By.XPATH,"//span[text()='READ MORE']"):
                try:
                    item.click()
                except ElementNotInteractableException:
                    pass
            
            # get the source code of the page
            soup = BeautifulSoup(driver.page_source, 'lxml')
            
            # find the reviews class and the store them in the variable
            for item in soup.find_all('div', {"class": "qwjRop"}):
                 Reviews = Reviews+'\n'+(str(item.getText(separator=u' ')))
                    
            # to go to the next page click on NEXT
            element = driver.find_element(By.LINK_TEXT, "NEXT")
            # you can do this using ActionChains
            ActionChains(driver).move_to_element(element).click(element).perform()
            # give time load the next page
            time.sleep(20)

        # last page will not have NEXT button so then break the loop
        except NoSuchElementException:
            break
            
    # append the details of the product in our list
    my_list_phone.append([Title,Price,Specifications,Ratings,Reviews])
    # close the tab
    driver.close()
    # switch back to primary window to repeat the same process
    driver.switch_to_window(handles[0])

# at the end close the chrome window
driver.quit()    

KeyboardInterrupt: 

In [186]:
# check the products you got reviews of
for i in range(0,len(my_list_phone)):
    print(my_list_phone[i][0])

Samsung Galaxy M11 (Violet, 32 GB)  (3 GB RAM)
Samsung Galaxy M11 (Violet, 64 GB)  (4 GB RAM)
Samsung Galaxy M11 (Black, 64 GB)  (4 GB RAM)
Infinix Hot 9 Pro (Ocean Wave, 64 GB)  (4 GB RAM)
Realme Narzo 10A (So Blue, 64 GB)  (4 GB RAM)
Vivo Y91i (Black, 32 GB)  (2 GB RAM)
Realme Narzo 10A (So Blue, 64 GB)  (4 GB RAM)
Vivo Y50 (Pearl White, 128 GB)  (8 GB RAM)
Samsung Galaxy A31 (Prism Crush Black, 128 GB)  (6 GB RAM)
OPPO K1 (Astral Blue, 64 GB)  (6 GB RAM)
Panasonic ELUGA I6 (Black, 16 GB)  (2 GB RAM)
Samsung Galaxy A21s (Blue, 64 GB)  (6 GB RAM)
Samsung Galaxy M11 (Black, 32 GB)  (3 GB RAM)
Realme 6 (Comet Blue, 128 GB)  (8 GB RAM)
Realme 6 Pro (Lightning Orange, 128 GB)  (8 GB RAM)
Realme C3 (Blazing Red, 64 GB)  (4 GB RAM)
Realme C3 (Blazing Red, 32 GB)  (3 GB RAM)


In [187]:
# store the reviews in a text file
with open('review_file_phone.txt', 'w',encoding="utf-8") as f:
    for item in my_list_phone:
        f.write(str(item))